In [3]:
#notebook to download the csv of edges and nodes of a given network
import os
import requests 
import time
import string
import networkx as nx
import itertools
import networkx as nx
import pandas as pd
import json
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

from nltk.corpus import wordnet as wn #importing it
from nltk.stem.wordnet import WordNetLemmatizer #importing wordnet lemmatizer
from nltk import pos_tag #part-of-speech-tagger
from collections import defaultdict #defaultdict returns default value for non-existant keys you try to  access based on the function you passed in the constructor
from google.colab import files

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def extract_text(df):       #extract the text from the tweets and RT
                            #works ONLY on .csv file
    list_strings = []
    for index in range(len(df)):
        #print(index)
        text = df.loc[index]['text']                          #if it is nor trucated nor a RT  i take "text"
        string = -1
        if (df.loc[index,"truncated"] == True):                 #if it is trucated I take "extended_tweet"
            string = df.loc[index,"extended_tweet"]
        if type(df.loc[index,"retweeted_status"]) != float:     #if it is a RT I take retweeted_status
            string = df.loc[index,"retweeted_status"]
        if type(string) == str :
            if(re.search('full_text\':(.+?)https',string) != None):     #if I find "full_text"
                s = re.search('full_text\':(.+?)https',string).group(1)
            if(re.search('text\':(.+?)https',string)!= None):
                s = re.search('text\':(.+?)https',string).group(1)
            else:  #here then one should remove the http
                if (re.search('full_text\':\s\'(.+?)\'',string)!=None) :
                    s=re.search('full_text\':\s\'(.+?)\'',string).group(1)  
                if (re.search('\{\\\'full_text\\\':\s(.+?)\"',string)!=None) : 
                    re.search('\{\\\'full_text\\\':\s(.+?)\"',string).group(1)
            
            list_strings.append(s)
            #print(s)         
        else:
            list_strings.append(text)
            print(text)
      

    return list_strings

In [6]:
# Cleaning, lemmatising and pos tagging tweets

nltk.download('words')
WORDS = set(nltk.corpus.words.words()) #the last two lines serve to download the corpus of standard English language words
nltk.download('stopwords') #downloading stopwords
STOP_WORDS = set(nltk.corpus.stopwords.words("english")) #taking the stop words from English language
nltk.download('wordnet') #downloading wordnet
nltk.download('averaged_perceptron_tagger') #downloading tagger
tag_map = defaultdict(lambda : wn.NOUN) #here we define that wn.NOUN is the default value for the dict
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def lemma_pos_cleaner(tweet):

    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase 
    tweet = re.sub("'"," ",tweet) # remove aphostrophe

    #basically we use pos_tag function on tokens that we get by applying wordpunct tokenization
    #to tweet (it separates all the words and symbols)
    #then we pass the token along with it's wordnet pos value that we get from the tag_map dictionary (noun, adjective, verb or adverb) to the lemma function (the WordNetLemmatizer())
    lemma_function = WordNetLemmatizer()
    tweet = " ".join(lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in nltk.pos_tag(nltk.wordpunct_tokenize(tweet))) #lemmatize
  

    # francesco: I removed also all 2 letters words and added specific words, words that appears frequently but are discarded because they are not in the english language
    SPECIFIC_WORDS = ['virus', 'coronavirus', 'covid19', 'covid', 'trump', 'hubei', 'beijing', 'xinjiang', 'jinping', 'korea', 'xinhua', 'india', 'taiwan','johnson','singapore', 'africa', 'japanese', 'france', 'asian', 'australia', 'french', 'asia', 'leishenshan', 'british', 'qingdao', 'fauci', 'america',  'california', 'sichuan', 'malaysia', 'huawei','thailand', 'shandong', 'italy', 'philippines', 'germany', 'facebook', 'african', 'shenzhen', 'tokyo', 'russian','uygur', '5g', 'pompeo', 'vietnam', 'australian', 'cambodia', 'zhejiang', 'yunnan', 'guangdong', 'korean', 'iran', 'washington']
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if (w in WORDS or w in SPECIFIC_WORDS) and len(w)>2 and w not in STOP_WORDS ) #remove stop words
   
    return tweet

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [7]:
def frequency_dictionary(df):
  unique_words = {}

  for row in df:
    for word in row.split():
      #if the word is encountered for the first time add to dict as key and set its value to 0
      unique_words.setdefault(word,0)
      #increase the value (i.e the count) of the word by 1 every time it is encountered
      unique_words[word] += 1

  return unique_words

In [206]:
period = '_MarchApril2021'  # '', '_JanFeb2020', '_MarchApril2021', '_SeptOct2020'

In [207]:
China = pd.read_csv('/content/China'+period+'.csv')
USA = pd.read_csv('/content/USA'+period+'.csv')
China_USA = pd.read_csv('/content/China&USA'+period+'.csv')

In [208]:
# number of tweets:
print('China: ', len(China))
print('USA: ', len(USA))
print('China&USA: ', len(China_USA))

China:  1330
USA:  3932
China&USA:  5262


In [209]:
text_China = extract_text(China)
text_USA = extract_text(USA)
text_China_USA = extract_text(China_USA)

Output streaming troncato alle ultime 5000 righe.
https://t.co/dEoMmBuLqj
China administers over 149 million COVID-19 vaccine doses https://t.co/OHVkjZkTDf https://t.co/YF6CdDqtSk
Italy, Britain suggest age limits for #AstraZeneca vaccine but still recommend it https://t.co/ztT0xQEmcT
Chile approves CanSino COVID-19 vaccine for emergency use https://t.co/HvNTeTtUGB https://t.co/AXBodB66PT
Belgium restricts AstraZeneca vaccine to over-55s, according to the health ministry. https://t.co/SkbF3rZzHm
EU drug regulator finds link between AstraZeneca vaccine and blood clots https://t.co/LEAIyty7bI
Over 500,000 #HongKong SAR residents have taken the first jab of the COVID-19 vaccine, official data showed. https://t.co/KLOx1BtQ1p
#Spain's Castile and Leon region suspends use of AstraZeneca #COVID19 vaccine, according to local media. https://t.co/ZBAadFJner
Oxford pauses #COVID19 vaccine study in kids, awaits more data on blood clot issues #AstraZenaca 
https://t.co/b4bvJ0Otl8
Watch: Serbian pre

In [210]:
cleaned_text_China = [lemma_pos_cleaner(txt) for txt in text_China]
cleaned_text_USA = [lemma_pos_cleaner(txt) for txt in text_USA]
cleaned_text_China_USA = [lemma_pos_cleaner(txt) for txt in text_China_USA]

print('China:')
print(cleaned_text_China[0:10])
print()
print('USA:')
print(cleaned_text_USA[0:10])
print()
print('China&USA:')
print(cleaned_text_China_USA[0:10])

China:
['death beloved rapper protest killing black america anguish lose lost one covid even historic celebration america pick photo week', 'german chancellor receive first dose covid vaccine announce via government spokesman people daily', 'number suicide drop study find among black', 'total covid jab give far close vaccine administer hour period accord government data', 'would like see covid passport infringe people freedom health passport claim', 'lift ban flight continue easing covid restriction', 'covid pandemic result permanent closure around extra business compare', 'german chancellor receive first dose vaccine', 'report new confirm case', 'million receive least one dose covid vaccine drug develop']

USA:
['decoration fill street old city welcome holy month freer covid', 'migrant worker pile rail station india head back home village covid curb', 'health economics education world change since covid', 'decoration fill street old city welcome holy month freer covid restriction heig

In [211]:
freq_dict_China = frequency_dictionary(cleaned_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  2012
USA:  3330
China&USA:  3853


In [213]:
# Most frequent words
print('China')
print([key for key in freq_dict_China.keys() if freq_dict_China[key]>100])
print()
print('USA')
print([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>100])
print()
print('China&USA')
print([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>100])

China
['covid', 'vaccine', 'china', 'say', 'case', 'country', 'receive', 'health', 'report', 'first', 'new', 'million', 'dos', 'people', 'pandemic', 'batch']

USA
['covid', 'vaccine', 'say', 'new', 'case', 'johnson', 'shot', 'million', 'use', 'report', 'people', 'get', 'india', 'country', 'first', 'health', 'dos', 'blood', 'vaccination', 'clot', 'death', 'coronavirus', 'pandemic', 'state', 'year', 'brazil', 'surge', 'president', 'world', 'infection', 'day', 'risk', 'record', 'trial', 'may', 'china', 'receive', 'official', 'rise', 'month', 'variant', 'vaccinate', 'high', 'study', 'plan', 'one', 'test', 'need', 'rare', 'week', 'make', 'could', 'amid']

China&USA
['covid', 'vaccine', 'say', 'new', 'case', 'china', 'report', 'million', 'country', 'first', 'health', 'people', 'johnson', 'dos', 'pandemic', 'use', 'receive', 'shot', 'get', 'vaccination', 'death', 'state', 'blood', 'clot', 'coronavirus', 'india', 'year', 'brazil', 'president', 'world', 'day', 'infection', 'surge', 'one', 'tria

# Build Network

In [214]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         covid    968
1       vaccine    779
2         china    313
3           say    190
4          case    174
5       country    169
6       receive    164
7        health    164
8        report    164
9         first    160
10          new    159
11      million    152
12          dos    122
13       people    119
14     pandemic    114
15        batch    105
16        world     84
17    president     78
18         dose     74
19          use     71
20       accord     69
21     national     68
22          day     67
23  vaccination     66
24       brazil     64
25         year     64
26          one     63
27        death     62
28        state     62
29       second     62

USA
           Word  Count
0         covid   3006
1       vaccine   2114
2           say    869
3           new    412
4          case    392
5       johnson    298
6          shot    258
7       million    257
8           use    249
9        report    246
10       people    246


In [215]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [216]:
def create_network(cleaned_text):
  network = {}
  #connect the word that appear in the same tweets
  for row in cleaned_text:
    combined_list = [word for word in str.split(row)]
    #for pair in itertools.product(combined_list, combined_list):
    #print(combined_list)
    for pair in itertools.product(combined_list, combined_list):
          #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
          if pair[0]!=pair[1] and not(pair[::-1] in network):
              network.setdefault(pair,0)
              network[pair] += 1 
  network_df = pd.DataFrame.from_dict(network, orient="index")
  network_df.columns = ["weight"]
  network_df.sort_values(by="weight",inplace=True, ascending=False)
  return network, network_df

In [217]:
network_China, network_df_China = create_network(cleaned_text_China)
network_USA, network_df_USA = create_network(cleaned_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_text_China_USA)

In [218]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                    weight
(covid, vaccine)       535
(vaccine, china)       254
(china, covid)         243
(receive, vaccine)     176
(covid, case)          147
(vaccine, say)         145
(million, covid)       142
(country, vaccine)     141
(covid, say)           137
(new, covid)           135
(dos, vaccine)         134
(covid, health)        132
(first, vaccine)       129
(covid, country)       128
(first, covid)         123
(covid, report)        122
(receive, covid)       122
(million, vaccine)     113
(vaccine, health)      110
(batch, vaccine)       109
(new, case)            108
(covid, pandemic)      108
(dos, covid)           104
(covid, people)         92
(batch, covid)          90
(million, dos)          89
(use, vaccine)          85
(report, case)          81
(dose, vaccine)         77
(vaccine, people)       71

USA:
                      weight
(covid, vaccine)        1450
(covid, say)             641
(vaccine, say)           626
(new, covid)             362
(covi

#Graph


In [187]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [219]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [220]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  2011
Edges:  42283
Is connected:  True

USA:
Nodes:  3330
Edges:  98919
Is connected:  True

China&USA:
Nodes:  3852
Edges:  128274
Is connected:  True


# PageRank

In [221]:
# Calculating the pagerank on graph G, teleportation probability here is 0.15 but since the graph is strongly connected we can set it to zero if we want
pr_China = nx.algorithms.pagerank(G_China,alpha = 1)
pr_China = dict(sorted(pr_China.items(), key=lambda item: item[1],reverse  = True))

pr_USA = nx.algorithms.pagerank(G_USA,alpha = 1)
pr_USA = dict(sorted(pr_USA.items(), key=lambda item: item[1],reverse  = True))

pr_China_USA = nx.algorithms.pagerank(G_China_USA,alpha = 1)
pr_China_USA = dict(sorted(pr_China_USA.items(), key=lambda item: item[1],reverse  = True))

In [26]:
def threshold(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] >= threshold ]

  return pd.DataFrame(l)

In [27]:
def threshold_reverse(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] < threshold ]

  return pd.DataFrame(l)

In [222]:
thr = 0.0004
print('China: ', len(threshold(pr_China,thr)))
print()
print(threshold(pr_China,thr).iloc[:30])
print()
print('USA: ', len(threshold(pr_USA,thr)))
print()
print(threshold(pr_USA,thr).iloc[:30])
print()
print('China&USA: ', len(threshold(pr_China_USA,thr)))
print()
print(threshold(pr_China_USA,thr).iloc[:30])

China:  485

              0         1
0         covid  0.058555
1       vaccine  0.044971
2         china  0.019651
3           say  0.014622
4       country  0.012645
5        health  0.012337
6          case  0.010970
7        report  0.010898
8         first  0.010215
9           new  0.009982
10      million  0.009683
11      receive  0.009466
12       people  0.008554
13     pandemic  0.007990
14          dos  0.007793
15        world  0.006431
16          day  0.005114
17       accord  0.005086
18        batch  0.004975
19          use  0.004909
20         year  0.004885
21    president  0.004861
22     national  0.004855
23        state  0.004814
24  vaccination  0.004628
25       brazil  0.004413
26         dose  0.004397
27        death  0.004327
28         city  0.004182
29   government  0.003902

USA:  512

              0         1
0         covid  0.057114
1       vaccine  0.041153
2           say  0.020410
3           new  0.008660
4          case  0.008049
5       johns

# TF-IDF

In [223]:
tfidf = TfidfVectorizer(ngram_range=(1,1))   # ngram range can be changed to obtain measures regarding n grams instead of single words

X_China = tfidf.fit_transform(cleaned_text_China).toarray()    # entry (i,j) if Tfidf measure of word_list[j] in document i
word_list_China = tfidf.get_feature_names_out()

X_USA = tfidf.fit_transform(cleaned_text_USA).toarray()
word_list_USA = tfidf.get_feature_names_out()

X_China_USA = tfidf.fit_transform(cleaned_text_China_USA).toarray()
word_list_China_USA = tfidf.get_feature_names_out()


In [224]:
tfidf_df_China = pd.DataFrame(X_China,columns = word_list_China)

tfidf_df_USA = pd.DataFrame(X_USA,columns = word_list_USA)

tfidf_df_China_USA = pd.DataFrame(X_China_USA,columns = word_list_China_USA)

In [225]:
tfidf_word_measure_China = np.mean(tfidf_df_China,axis = 0)
tfidf_word_measure_China = tfidf_word_measure_China.sort_values(ascending = False)
tfidf_word_measure_USA = np.mean(tfidf_df_USA,axis = 0)
tfidf_word_measure_USA = tfidf_word_measure_USA.sort_values(ascending = False)
tfidf_word_measure_China_USA = np.mean(tfidf_df_China_USA,axis = 0)
tfidf_word_measure_China_USA = tfidf_word_measure_China_USA.sort_values(ascending = False)

In [226]:
print('China:')
print(tfidf_word_measure_China[0:30])
print()
print('USA:')
print(tfidf_word_measure_USA[0:30])
print()
print('China&USA:')
print(tfidf_word_measure_China_USA[0:30])
print()

China:
vaccine        0.071675
covid          0.069441
china          0.042245
receive        0.030198
new            0.028422
case           0.028334
first          0.027963
million        0.026889
batch          0.026760
report         0.026149
dos            0.023979
say            0.023827
country        0.023254
health         0.023176
people         0.018490
watch          0.018144
pandemic       0.016410
dose           0.016282
president      0.015292
second         0.015083
arrive         0.014773
use            0.013589
world          0.013443
accord         0.013282
one            0.013259
administer     0.013204
national       0.012990
vaccination    0.012374
brazil         0.011739
death          0.011625
dtype: float64

USA:
covid          0.065701
vaccine        0.062109
say            0.033557
new            0.023122
case           0.023037
million        0.020074
dos            0.018550
johnson        0.017527
report         0.016294
india          0.015845
use         

# reduced graph

In [227]:
# less important words:
less_important_words_China = list(threshold_reverse(pr_China,thr)[0])

less_important_words_USA = list(threshold_reverse(pr_USA,thr)[0])

less_important_words_China_USA = list(threshold_reverse(pr_China_USA,thr)[0])

In [197]:
def clean_words(cleaned_text, unuseful_words):
  re_cleaned_text = cleaned_text.copy()
  for txt in range(len(re_cleaned_text)):
    if txt % 2000 == 0:
      print(txt, '/',len(re_cleaned_text))
    w = re_cleaned_text[txt].split()
    for word in unuseful_words:
      while word in w:
        w.remove(word)
    re_cleaned_text[txt] = ' '.join(w)
  return re_cleaned_text

In [228]:
cleaned_mostimp_text_China = clean_words(cleaned_text_China,less_important_words_China)
cleaned_mostimp_text_USA = clean_words(cleaned_text_USA,less_important_words_USA)
cleaned_mostimp_text_China_USA = clean_words(cleaned_text_China_USA,less_important_words_China_USA)

0 / 1330
0 / 3932
2000 / 3932
0 / 5262
2000 / 5262
4000 / 5262


In [229]:
freq_dict_China = frequency_dictionary(cleaned_mostimp_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostimp_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostimp_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  486
USA:  512
China&USA:  501


In [230]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         covid    968
1       vaccine    779
2         china    313
3           say    190
4          case    174
5       country    169
6       receive    164
7        health    164
8        report    164
9         first    160
10          new    159
11      million    152
12          dos    122
13       people    119
14     pandemic    114
15        batch    105
16        world     84
17    president     78
18         dose     74
19          use     71
20       accord     69
21     national     68
22          day     67
23  vaccination     66
24       brazil     64
25         year     64
26          one     63
27        death     62
28        state     62
29       second     62

USA
           Word  Count
0         covid   3006
1       vaccine   2114
2           say    869
3           new    412
4          case    392
5       johnson    298
6          shot    258
7       million    257
8           use    249
9        report    246
10       people    246


In [231]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [232]:
network_China, network_df_China = create_network(cleaned_mostimp_text_China)
network_USA, network_df_USA = create_network(cleaned_mostimp_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostimp_text_China_USA)

In [233]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                    weight
(covid, vaccine)       535
(vaccine, china)       254
(china, covid)         243
(receive, vaccine)     176
(covid, case)          147
(vaccine, say)         145
(million, covid)       142
(country, vaccine)     141
(covid, say)           137
(new, covid)           135
(dos, vaccine)         134
(covid, health)        132
(first, vaccine)       129
(covid, country)       128
(first, covid)         123
(receive, covid)       122
(covid, report)        122
(million, vaccine)     113
(vaccine, health)      110
(batch, vaccine)       109
(new, case)            108
(covid, pandemic)      108
(dos, covid)           104
(covid, people)         92
(batch, covid)          90
(million, dos)          89
(use, vaccine)          85
(report, case)          81
(dose, vaccine)         77
(vaccine, people)       71

USA:
                      weight
(covid, vaccine)        1450
(covid, say)             641
(vaccine, say)           626
(new, covid)             362
(covi

In [44]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [234]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [235]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  485
Edges:  17993
Is connected:  True

USA:
Nodes:  512
Edges:  34347
Is connected:  True

China&USA:
Nodes:  500
Edges:  40122
Is connected:  True


#Save edge list

In [236]:
filename = './edgelist_China_MarchApril2021.csv'
nx.write_weighted_edgelist(G_China, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_MarchApril2021.csv
files.download("edgelist_China_MarchApril2021.csv")

filename = './edgelist_USA_MarchApril2021.csv'
nx.write_weighted_edgelist(G_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_USA_MarchApril2021.csv
files.download("edgelist_USA_MarchApril2021.csv")

filename = './edgelist_China_USA_MarchApril2021.csv'
nx.write_weighted_edgelist(G_China_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_USA_MarchApril2021.csv
files.download("edgelist_China_USA_MarchApril2021.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>





# Create Node List




In [48]:
def nodes(freq_dict, name):
  word_nodes = pd.DataFrame.from_dict(freq_dict,orient="index")
  word_nodes.reset_index(inplace=True)
  word_nodes["Label"] = word_nodes["index"]
  word_nodes.rename(columns={"index":"Id",0:"delete"},inplace=True)
  word_nodes = word_nodes.drop(columns=['delete'])
  nodelist = pd.DataFrame()
  nodelist = nodelist.append(word_nodes, ignore_index=True)

  nodelist = nodelist.to_csv("nodelist_"+name+".csv",index=False)
  files.download("nodelist_"+name+".csv")
  return nodelist, word_nodes

In [237]:
nodelist_China, word_nodes_China = nodes(freq_dict_China,"China_MarchApril2021")
nodelist_USA, word_nodes_USA = nodes(freq_dict_USA,"USA_MarchApril2021")
nodelist_China_USA, word_nodes_China_USA = nodes(freq_dict_China_USA,"China_USA_MarchApril2021")

print('China:')
print(word_nodes_China.head())
print()
print('USA:')
print(word_nodes_USA.head())
print()
print('China&USA:')
print(word_nodes_China_USA.head())
print()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

China:
        Id    Label
0    covid    covid
1  vaccine  vaccine
2    china    china
3      say      say
4     case     case

USA:
        Id    Label
0    covid    covid
1  vaccine  vaccine
2      say      say
3      new      new
4     case     case

China&USA:
        Id    Label
0    covid    covid
1  vaccine  vaccine
2      say      say
3      new      new
4     case     case

